In [3]:
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import json

# Load the CSV file
file_path = '/data1/dxw_data/llm/RA/hku_ivy/esg_variable/LIWC-22-Factiva_tw_final_match-TXT.csv'
data = pd.read_csv(file_path)

# Load and clean Brysbaert concreteness scores
brysbaert = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/NLP/LIWC/BCI/R_Functions/brysbaert.txt', sep='\t')
brysbaert['Conc.M'] = pd.to_numeric(brysbaert['Conc.M'], errors='coerce')
brysbaert = brysbaert.dropna(subset=['Conc.M'])

# Define the BCI calculation function
def brysbaert_calculator(texts, keep):
    scores = []
    for text in tqdm(texts, desc="计算BCI值"):
        words = str(text).lower().split()
        filtered_words = [word for word in words if word in keep]
        bryscore = sum(brysbaert.loc[brysbaert['Word'].isin(filtered_words), 'Conc.M'])
        word_count = len(filtered_words)
        if word_count > 0:
            bryscore /= word_count
        scores.append(bryscore)
    return scores

# Apply BCI calculation to post_content column
data['BCI_score'] = brysbaert_calculator(data['TXT'], brysbaert['Word'].tolist())

# Save the updated DataFrame back to the CSV file
data.to_csv(file_path, index=False)


计算BCI值: 100%|██████████| 1493/1493 [01:54<00:00, 12.99it/s]
